#Instal dependencies

In [1]:
!pip install pandas numpy tensorflow scikit-learn

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-12-24 22:51:27.200339: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-24 22:51:27.201103: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-24 22:51:27.204580: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-24 22:51:27.213197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735060887.226127  281064 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735060887.22

#Load Dataset

In [3]:
df = pd.read_csv("/home/navin-lakshh/Documents/DocUnderstand/shuffled_file.csv", encoding='latin-1')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1363 entries, 0 to 1362
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Label     1363 non-null   int64 
 1   Document  1363 non-null   object
dtypes: int64(1), object(1)
memory usage: 21.4+ KB


In [ ]:
df

,Label,Document
0,2,"The community of WEST HARTFORD (Tax Code 155),..."
1,2,"The town of PRESTON (Tax Code 114), with a pop..."
2,2,"The community of WEST HAVEN (Tax Code 156), wi..."
3,0,"I, Vijay Kumar, holding an NRI account with ac..."
4,0,"I, Arvind Kumar, request a credit card for mak..."
...,...,...
1358,1,USA Florida DRIVER LICENSE PH 754-898-9516 CLA...
1359,2,"The community of THOMPSON (Tax Code 141), with..."
1360,2,"The community of BOLTON (Tax Code 12), with a ..."
1361,1,PERMANENT RESIDENT CARD NAME CRITTENDEN LeeW I...


#Preprocessing

In [6]:
import re

In [7]:
def preprocessing(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

<>:11: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\)'
<>:13: SyntaxWarning: invalid escape sequence '\?'
<>:11: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\)'
<>:13: SyntaxWarning: invalid escape sequence '\?'
/tmp/ipykernel_281064/1229834350.py:11: SyntaxWarning: invalid escape sequence '\('
  string = re.sub(r"\(", " \( ", string)
/tmp/ipykernel_281064/1229834350.py:12: SyntaxWarning: invalid escape sequence '\)'
  string = re.sub(r"\)", " \) ", string)
/tmp/ipykernel_281064/1229834350.py:13: SyntaxWarning: invalid escape sequence '\?'
  string = re.sub(r"\?", " \? ", string)


In [8]:
df['Document'] = df['Document'].apply(preprocessing)

In [9]:
df

,Label,Document
0,2,the community of west hartford \( tax code 155...
1,2,"the town of preston \( tax code 114 \) , with ..."
2,2,the community of west haven \( tax code 156 \)...
3,0,"i , vijay kumar , holding an nri account with ..."
4,0,"i , arvind kumar , request a credit card for m..."
...,...,...
1358,1,usa florida driver license ph 754 898 9516 cla...
1359,2,"the community of thompson \( tax code 141 \) ,..."
1360,2,"the community of bolton \( tax code 12 \) , wi..."
1361,1,permanent resident card name crittenden leew i...


In [10]:
df['Document'][3]

'i , vijay kumar , holding an nri account with account number 123456789 , request an atm card for my account kindly process my request'

In [11]:
texts = df['Document'].values
labels = df['Label'].values

In [12]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=100)
y = labels

#Encode Labels

In [13]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
y

array([2, 2, 2, ..., 2, 1, 1])

#Train-test split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(y_test)
print(y_train)
print(y)


[3 0 3 3 3 2 3 2 0 3 0 2 2 0 1 0 1 2 2 2 0 0 3 2 3 2 2 0 2 2 3 0 1 2 2 1 0
 2 1 3 1 1 2 1 2 2 3 0 3 2 1 0 3 0 3 0 3 0 3 1 0 2 2 1 0 2 2 3 0 0 1 2 3 2
 0 2 0 2 2 3 3 3 1 1 1 3 3 2 2 1 2 2 1 0 1 2 0 3 2 0 2 2 3 2 1 3 3 1 2 2 3
 1 0 2 2 3 2 1 0 3 1 2 3 3 0 0 2 3 2 2 0 3 0 3 2 3 3 0 0 0 2 2 2 1 0 2 2 2
 0 1 0 0 2 0 1 0 0 2 0 2 2 2 3 1 2 2 0 2 3 0 2 2 2 2 3 1 3 2 2 2 1 2 2 1 2
 2 2 2 1 2 2 3 3 3 1 1 3 2 2 3 2 2 1 2 0 0 2 0 1 3 0 2 1 2 2 3 2 2 2 3 2 0
 2 1 3 2 2 2 0 2 0 1 0 0 0 3 1 3 0 2 0 3 2 2 1 2 1 0 3 3 2 1 2 0 3 2 3 0 3
 3 1 0 3 3 0 3 2 1 2 0 3 3 3]
[0 0 0 ... 2 1 2]
[2 2 2 ... 2 1 1]


#LSTM Model

In [15]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax')) # 4 classes

/home/navin-lakshh/miniconda3/envs/neuDocUnder/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-12-24 22:51:34.713420: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [16]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


#Train the model

In [17]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 396ms/step - accuracy: 0.4340 - loss: 1.3000 - val_accuracy: 0.6960 - val_loss: 0.7028
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 244ms/step - accuracy: 0.8018 - loss: 0.5548 - val_accuracy: 0.9267 - val_loss: 0.2395
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 253ms/step - accuracy: 0.9537 - loss: 0.2061 - val_accuracy: 0.9597 - val_loss: 0.1392
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 252ms/step - accuracy: 0.9826 - loss: 0.0866 - val_accuracy: 0.9780 - val_loss: 0.0787
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 257ms/step - accuracy: 0.9936 - loss: 0.0442 - val_accuracy: 0.9744 - val_loss: 0.0770
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 326ms/step - accuracy: 0.9965 - loss: 0.0294 - val_accuracy: 0.9524 - val_loss: 0.1379
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 351ms/step - accuracy: 0.9943 - loss: 0.0397 - val_accuracy: 0.9817 - val_loss: 0.0708
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 334ms/step - accuracy: 0.9979 - loss: 0.0197 - val_accuracy: 0.

#Save the model

In [19]:

model.save('lstm_model.h5')
print("Model saved to 'lstm.h5'.")

Model saved to 'lstm.h5'.


In [20]:
model.save('lstm_keras_model.keras')

#Save Toeknizers and labelEncoders

In [21]:
import pickle

In [22]:
with open("Tokenizer.pickle",'wb') as t:
    pickle.dump(tokenizer,t,protocol=pickle.HIGHEST_PROTOCOL)


In [28]:
with open("LabelEncoder.pickle",'wb') as lb:
    pickle.dump(label_encoder,lb,protocol=pickle.HIGHEST_PROTOCOL)

#Load the model

In [29]:
loaded_model = load_model('lstm_keras_model.keras')
print("Model loaded successfully.")

Model loaded successfully.


/home/navin-lakshh/miniconda3/envs/neuDocUnder/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [30]:
score = loaded_model.evaluate(X_test, y_test, batch_size=64)
print("Test accuracy: %.2f%%" % (score[1] * 100))

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step - accuracy: 0.9788 - loss: 0.0832
Test accuracy: 97.80%


#Prediction Function

In [31]:
def predict_label(input_text):
    # Preprocess the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_padded = pad_sequences(input_sequence, maxlen=100)

    # Predict the label
    prediction = loaded_model.predict(input_padded)
    predicted_class = np.argmax(prediction, axis=1)[0]
    label = label_encoder.inverse_transform([predicted_class])[0]

    return label

In [27]:
from flask import Flask, render_template, request, redirect, url_for
from tensorflow.keras.models import load_model
import numpy as np
import time
import os
import pdf2image as pdi
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import base64
from PIL import Image
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pickle
# from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize Flask app
app = Flask(__name__)
load_dotenv()


def encode(image):
    with open(image, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")
    
def t2i(image):
    bs64img = encode(image)

    client = OpenAI(
        api_key=os.getenv('XAI'),
        base_url="https://api.x.ai/v1",
    )

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{bs64img}",
                        "detail": "high",
                    },
                },
                {
                    "type": "text",
                    "text": "You are an OCR model. Your task is to extract only the English words from the attached image. Return only the extracted text. Do not provide any context, explanations, or other commentary.",
                },
            ],
        },
    ]

    completion = client.chat.completions.create(
        model="grok-2-vision-1212",
        messages=messages,
        stream=False,
        temperature=0.01,
    )

    return completion.choices[0].message.content    



# Function to extract text from PDF
def extract_text_from_pdf(pdf_file):
    pdf_images = pdi.convert_from_path(pdf_file, dpi=300)
    dir_path = os.getcwd()
    image_dir = os.path.join(dir_path, 'pdf_images')
    os.makedirs(image_dir, exist_ok=True)
    all_text = ""
    
    for i, img in enumerate(pdf_images):
        image_path = os.path.join(image_dir, f'page_{i}_{time.time()}.jpg')
        img.save(image_path, 'JPEG')
        result = t2i(image_path)
        # page_text = ' '.join([text[1] for text in result])
        all_text += result + '\n'
    print("Txt Extrt")
    print(all_text)
    print("Txt extrt done")
    return all_text

# Route to home page
@app.route('/')
def index():
    return render_template('index.html')

# Route to handle PDF upload and classification
@app.route('/classify', methods=['POST'])
def classify():
    if 'pdf_file' not in request.files:
        return redirect(request.url)
    
    pdf_file = request.files['pdf_file']
    
    if pdf_file.filename == '':
        return redirect(request.url)
    
    if pdf_file:
        # Save the uploaded PDF file
        pdf_path = os.path.join(os.path.dirname(__file__), 'uploads')
        os.makedirs(pdf_path, exist_ok=True)
        pdf_path = os.path.join(os.path.dirname(__file__), 'uploads', pdf_file.filename)
        pdf_file.save(pdf_path)

        # Extract text from PDF
        extracted_text = extract_text_from_pdf(pdf_path)

        print("Txt Extrt")
        print(extracted_text)
        print("Txt extrt done")
    

            # extracted_text = vectz.fit_transform([extracted_text])
        # Predict the label using the extracted text
        predicted_label = predict_label(extracted_text)
            # prediction=loaded_model.predict(extracted_text)
            # predicted_class = np.argmax(prediction, axis=1)[0]
            # predicted_label = label_encoder.inverse_transform([predicted_class])[0]
        print(predicted_label)
        # predicted_label=np.argmax(predicted_label,axis=1)[0]
        # print(predicted_label)
        # Display results
        if predicted_label == 0:
            document_type = "Application Document"
        elif predicted_label == 1:
            document_type = "Identity Document"
        elif predicted_label == 2:
            document_type = "Financial Document"
        elif predicted_label == 3:
            document_type = "Receipt"
        else:
            document_type = "None"

        return render_template('index.html', document_type=document_type)

# Run the app
if __name__ == '__main__':
    app.run(debug=True,port=5001)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/navin-lakshh/miniconda3/envs/neuDocUnder/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/navin-lakshh/miniconda3/envs/neuDocUnder/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/navin-lakshh/miniconda3/envs/neuDocUnder/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/navin-lakshh/miniconda3/envs/neuDocUnder/lib/python3.12

SystemExit: 1

/home/navin-lakshh/miniconda3/envs/neuDocUnder/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [25]:
# Get user input and predict
while True:
    user_input = input("Enter text (or 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    predicted_label = predict_label(user_input)
    print(f"Predicted label: {predicted_label}")
print('ko')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
Predicted label: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Predicted label: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Predicted label: 2
ko


In [26]:
ll=predict_label(" The community of ANSONIA (Tax Code 2), with a population of 18923 as of 2022, reports a fiscal year-end fund balance of $3817995.00 in unassigned funds and $7982841.00 in total fund balance deficit. The unemployment rate averaged 5.88% for the year. The community holds a net pension liability of $16179257.00 and a net OPEB liability of $231882689.00, with bonded long-term debt amounting to $30411026.00. The annual debt service is recorded at $2301205.00. The equalized net grand list valuation is $1862158093.00.")
ll.tolist()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step


2

#Testing

In [ ]:
if predicted_label == 0:
  print("Application Document")
elif predicted_label == 1:
  print("Identity Document")
elif predicted_label == 2:
  print("Finiacnial Document")
elif predicted_label == 3:
  print("Receipt")
else:
  print("None")